In [ ]:
import os
%pwd



In [ ]:


os.chdir("../")
%pwd



In [ ]:


from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    dataset_name: str
    raw_dataset_dir: Path




In [ ]:


from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self, 
                config_path = CONFIG_FILE_PATH, 
                params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            dataset_name = config.dataset_name,
            raw_dataset_dir = config.raw_dataset_dir,
        )
        return data_ingestion_config




In [ ]:


import os
from datasets import load_dataset
from src.textSummarizer.logging import logger


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def fetch_and_save_dataset(self):
        logger.info("Loading dataset from Hugging Face...")
        dataset = load_dataset(self.config.dataset_name)

        logger.info("Saving raw dataset locally...")
        dataset.save_to_disk(self.config.raw_dataset_dir)
        
        logger.info("Saving dataset splits locally...")
        dataset['train'].to_csv(os.path.join(self.config.root_dir, "samsum-train.csv"), index=False)
        dataset['test'].to_csv(os.path.join(self.config.root_dir, "samsum-test.csv"), index=False)
        dataset['validation'].to_csv(os.path.join(self.config.root_dir, "samsum-validation.csv"), index=False)

        logger.info("Data ingestion from Hugging Face completed.")




In [ ]:


config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)

data_ingestion.fetch_and_save_dataset()




In [ ]:


import os
%pwd



In [ ]:


os.chdir("../")
%pwd



In [ ]:


%pwd



In [ ]:


from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path




In [ ]:


from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories



In [ ]:


class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_path)
        self.paramss = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )
        return data_transformation_config





In [ ]:


import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk, DatasetDict

class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    def convert_examples_to_features(self, example_batch):
        dialogues = [d if isinstance(d, str) else "" for d in example_batch['dialogue']]
        summaries = [s if isinstance(s, str) else "" for s in example_batch['summary']]

        input_encodings = self.tokenizer(dialogues, max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(summaries, max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)

        # Apply conversion split-by-split
        dataset_samsum_pt = {}
        for split in dataset_samsum.keys():
            logger.info(f"Processing split: {split}")
            dataset_samsum_pt[split] = dataset_samsum[split].map(
                self.convert_examples_to_features,
                batched=True
            )

        dataset_samsum_pt = DatasetDict(dataset_samsum_pt)
        
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))





In [ ]:


config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTransformation(config=data_transformation_config)
data_transformation.convert()




In [ ]:


import os
%pwd



In [ ]:


os.chdir("../")
%pwd



In [ ]:


from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int




In [ ]:


from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories



In [ ]:


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) 


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            eval_strategy = params.eval_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config





In [ ]:


from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk



In [ ]:


class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            eval_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))






In [ ]:


!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate



In [ ]:


config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

